### Subset scRNAseq data from matched individuals for label transfer to scATACseq

In [23]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import os

import sys

def MovePlots(plotpattern, subplotdir):
    os.system('mkdir -p '+str(sc.settings.figdir)+'/'+subplotdir)
    os.system('mv '+str(sc.settings.figdir)+'/*'+plotpattern+'** '+str(sc.settings.figdir)+'/'+subplotdir)

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.figdir = '/home/jovyan/MULTIOME_july2021/figures_males/'
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

sys.executable

-----
anndata     0.7.5
scanpy      1.7.0
sinfo       0.3.1
-----
PIL                 8.1.0
anndata             0.7.5
backcall            0.2.0
cairo               1.20.0
cffi                1.14.4
constants           NA
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
decorator           4.4.2
get_version         2.1
h5py                3.1.0
highs_wrapper       NA
igraph              0.8.3
ipykernel           5.4.3
ipython_genutils    0.2.0
jedi                0.18.0
joblib              1.0.0
kiwisolver          1.3.1
legacy_api_wrap     1.2
leidenalg           0.8.3
llvmlite            0.35.0
matplotlib          3.3.4
mpl_toolkits        NA
natsort             7.1.1
numba               0.52.0
numexpr             2.7.2
numpy               1.21.2
packaging           20.9
pandas              1.2.1
parso               0.8.1
pexpect             4.8.0
pickleshare         0.7.5
pkg_resources       NA
prompt_toolkit      3.0.14
ptyprocess          0.7.0
pygments  

'/opt/conda/envs/atac_env/bin/python'

#### Load scRNAseq data

In [24]:
adata = sc.read('/nfs/users/nfs_l/lg18/team292/lg18/with_valentina/gonadsV2_revision/FCA-gonads_rawcounts.h5ad')
metadata_males = pd.read_csv('/nfs/users/nfs_l/lg18/team292/lg18/with_valentina/gonadsV2_revision/FCA-gonads_male_annotated.csv', index_col = 0)

/opt/conda/envs/atac_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
metadata_males.head()

,n_genes,sample,donor,location,stage,sex,study,batch_collection,enrichment,cryopreserved,...,lineages_v1,celltype_v1,lineage_v2,somatic_celltype_v2,PCW,sample_source,trimester,leiden,leiden_R,lineage
FCA_GND8289580_AAACCTGAGACAGGCT,2696,FCA_GND8289580,F88,Gonad_and_extragonadal,11.0,male,FCA,D,CD45-,No,...,Mesenchymal_LHX9_ARX,Ti,Mesenchymal_LHX9,Ti,11.0,N,1st,0,0,Mesenchymal_LHX9
FCA_GND8289580_AAACCTGAGACGCACA,1591,FCA_GND8289580,F88,Gonad_and_extragonadal,11.0,male,FCA,D,CD45-,No,...,Mesenchymal_LHX9_ARX,Ti,Mesenchymal_LHX9,Ti,11.0,N,1st,0,0,Mesenchymal_LHX9
FCA_GND8289580_AAACCTGAGAGTCGGT,1383,FCA_GND8289580,F88,Gonad_and_extragonadal,11.0,male,FCA,D,CD45-,No,...,Coel Epi,coelEpi,CoelEpi,CoelEpi_LHX9,11.0,N,1st,10,10,CoelEpi
FCA_GND8289580_AAACCTGAGCCCTAAT,1365,FCA_GND8289580,F88,Gonad_and_extragonadal,11.0,male,FCA,D,CD45-,No,...,Sertoli,Sertoli,Sertoli,Sertoli,11.0,N,1st,2,2,Sertoli
FCA_GND8289580_AAACCTGAGCTTCGCG,1221,FCA_GND8289580,F88,Gonad_and_extragonadal,11.0,male,FCA,D,CD45-,No,...,Sertoli,Sertoli,Sertoli,Sertoli,11.0,N,1st,2,2,Sertoli


In [26]:
metadata_males['lineage'].value_counts(dropna = False)

Mesenchymal_LHX9     38177
Sertoli              34893
Mesenchymal_GATA2    33036
lowQC                11046
Endothelial           6123
Doublet               4156
Supporting            3959
GermCells             3608
Immune                2942
CoelEpi               2877
FetalLeydig           2772
Epithelial            2716
Erythroid             2622
PV                    2056
Neural                1610
Name: lineage, dtype: int64

In [27]:
mapping = metadata_males['lineage'].to_dict()
adata.obs['lineage'] = adata.obs.index.map(mapping)

In [28]:
adata.obs['lineage'].value_counts(dropna = False)

NaN                  243972
Mesenchymal_LHX9      38177
Sertoli               34893
Mesenchymal_GATA2     33036
lowQC                 11046
Endothelial            6123
Doublet                4156
Supporting             3959
GermCells              3608
Immune                 2942
CoelEpi                2877
FetalLeydig            2772
Epithelial             2716
Erythroid              2622
PV                     2056
Neural                 1610
Name: lineage, dtype: int64

In [29]:
np.unique(adata.obs['donor'])

array(['F100', 'F122', 'F123', 'F126', 'F133', 'F81', 'F83', 'F84', 'F86',
       'F87', 'F88', 'F89', 'F91', 'F93', 'F96', 'F99', 'Hrv10', 'Hrv11',
       'Hrv12', 'Hrv13', 'Hrv15', 'Hrv16', 'Hrv17', 'Hrv18', 'Hrv21',
       'Hrv25', 'Hrv27', 'Hrv3', 'Hrv30', 'Hrv31', 'Hrv32', 'Hrv33',
       'Hrv34', 'Hrv37', 'Hrv39', 'Hrv40', 'Hrv41', 'Hrv47', 'Hrv49',
       'Hrv51', 'Hrv54', 'Hrv55', 'Hrv57', 'Hrv58', 'Hrv59', 'Hrv85',
       'Hrv86', 'Hrv91', 'Hrv92', 'Hrv93', 'Hrv99'], dtype=object)

In [30]:
adata = adata[[i in ['Hrv10', 'Hrv15', 'Hrv17', 'Hrv18', 'Hrv21', 'Hrv3', 'Hrv40',
       'Hrv41', 'Hrv54', 'Hrv55'] for i in adata.obs['donor']]]
adata.shape

/opt/conda/envs/atac_env/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


(69917, 28820)

In [31]:
adata.obs['lineage'].value_counts(dropna = False)

Mesenchymal_GATA2    18099
Sertoli              12874
Mesenchymal_LHX9     12350
lowQC                 9626
Endothelial           2334
Supporting            2268
Epithelial            2115
Immune                1646
Neural                1579
CoelEpi               1570
GermCells             1558
FetalLeydig           1531
PV                    1053
Erythroid              695
Doublet                619
Name: lineage, dtype: int64

In [32]:
adata = adata[[i not in ['lowQC', 'Doublet'] for i in adata.obs['lineage']]]
adata.obs['lineage'].value_counts(dropna = False)

Mesenchymal_GATA2    18099
Sertoli              12874
Mesenchymal_LHX9     12350
Endothelial           2334
Supporting            2268
Epithelial            2115
Immune                1646
Neural                1579
CoelEpi               1570
GermCells             1558
FetalLeydig           1531
PV                    1053
Erythroid              695
Name: lineage, dtype: int64

#### Downsample 

In [33]:
myindex = adata.obs['lineage'].value_counts().index 
myvalues = adata.obs['lineage'].value_counts().values
clusters = pd.Series(myvalues, index = myindex)
clusters.values

array([18099, 12874, 12350,  2334,  2268,  2115,  1646,  1579,  1570,
        1558,  1531,  1053,   695])

In [34]:
import random
from itertools import chain

# Find clusters with > n cells
n = 8000
cl2downsample = clusters.index[ clusters.values > n ]

# save all barcode ids from small clusters
holder = []
holder.append( adata.obs_names[[ i not in cl2downsample for i in adata.obs['lineage'] ]] ) 

# randomly sample n cells in the cl2downsample
for cl in cl2downsample:
    print(cl)
    cl_sample = adata[[ i == cl for i in adata.obs['lineage'] ]].obs_names
    # n = int(round(len(cl_sample)/2, 0))
    cl_downsample = random.sample(set(cl_sample), n )
    holder.append(cl_downsample)
    
# samples to include
samples = list(chain(*holder))

# Filter adata_count
adata = adata[[ i in samples for i in adata.obs_names ]]
adata.X.shape

Mesenchymal_GATA2
Sertoli
Mesenchymal_LHX9


(40349, 28820)

In [35]:
adata

View of AnnData object with n_obs × n_vars = 40349 × 28820
    obs: 'n_genes', 'sample', 'donor', 'location', 'stage', 'sex', 'study', 'batch_collection', 'enrichment', 'cryopreserved', 'TP', '10xKit', 'dig_protocol', 'percent_mito', 'n_counts', 'batch', 'low_ncounts', 'high_mito', 'low_ncounts_high_mito', 'S_score', 'G2M_score', 'phase', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'is_doublet', 'lineages_v1', 'celltype_v1', 'lineage_v2', 'somatic_celltype_v2', 'PCW', 'sample_source', 'trimester', 'lineage'
    var: 'gene_ids-0', 'feature_types-0', 'n_cells-0', 'gene_ids-1', 'feature_types-1', 'n_cells-1', 'gene_ids-10', 'feature_types-10', 'n_cells-10', 'gene_ids-11', 'feature_types-11', 'n_cells-11', 'gene_ids-12', 'feature_types-12', 'n_cells-12', 'gene_ids-13', 'feature_types-13', 'n_cells-13', 'gene_ids-14', 'feature_types-14', 'n_cells-14', 'gene_ids-15', 'feature_types-15', 'n_cells-15', 'gene_ids-16', 'feature_types-16', 'n_cells-16', 'gene_ids

#### Save anndata object to perform integration with ATAC with Seurat

In [36]:
del adata.uns 

In [37]:
for i in adata.obs.columns:
    if i not in ['lineage', 'donor', 'sample']:
        del adata.obs[i] 

In [38]:
adata.obs

,sample,donor,lineage
FCA_GND8715519_AAACCTGCACAGAGGT,FCA_GND8715519,Hrv3,Immune
FCA_GND8715519_AAACCTGCACGAGGTA,FCA_GND8715519,Hrv3,Erythroid
FCA_GND8715519_AAACCTGCATGGGACA,FCA_GND8715519,Hrv3,GermCells
FCA_GND8715519_AAACCTGTCCCTAATT,FCA_GND8715519,Hrv3,Immune
FCA_GND8715519_AAACCTGTCCGTAGGC,FCA_GND8715519,Hrv3,GermCells
...,...,...,...
HD_F_GON9883868_TTTGGTTAGTGTACTC,HD_F_GON9883868,Hrv54,Sertoli
HD_F_GON9883868_TTTGGTTCACAACTGT,HD_F_GON9883868,Hrv54,Sertoli
HD_F_GON9883868_TTTGGTTGTTCAGGCC,HD_F_GON9883868,Hrv54,Sertoli
HD_F_GON9883868_TTTGTCAGTAAAGGAG,HD_F_GON9883868,Hrv54,Mesenchymal_GATA2


In [39]:
for j in adata.var.columns:
    if j != 'gene_ids-0':
        del adata.var[j]

In [40]:
adata.var

,gene_ids-0
A1BG,ENSG00000121410
A1BG-AS1,ENSG00000268895
A1CF,nan
A2M,ENSG00000175899
A2M-AS1,ENSG00000245105
...,...
ZXDC,ENSG00000070476
ZYG11A,ENSG00000203995
ZYG11B,ENSG00000162378
ZYX,ENSG00000159840


In [41]:
adata

AnnData object with n_obs × n_vars = 40349 × 28820
    obs: 'sample', 'donor', 'lineage'
    var: 'gene_ids-0'

In [42]:
# Define variables
outdir = "/nfs/team292/vl6/my_MULTIOME_dir/males_sept2021/"
experiment_prefix = 'males_'

In [43]:
adata.write(outdir + experiment_prefix + "_RNAseq.h5ad")

... storing 'lineage' as categorical
